In [27]:
with open('input.txt', 'r') as f:
    a, b = f.read().split()


In [5]:
from collections import defaultdict

def parse_path(string):
    horizontals = defaultdict(list)
    verticals = defaultdict(list)
    path = string.split(',')
    x0, y0, x1, y1 = [0] * 4
    for step in path:
        d = step[0]
        val = int(step[1:])
        if d == 'R':
            x1 = x0 + val
        elif d == 'U':
            y1 = y0 + val
        elif d == 'L':
            x1 = x0 - val
        elif d == 'D':
            y1 = y0 - val
        else:
            raise Exception('unknown direction')
        if y0 == y1:
            horizontals[y0].append((min(x0,x1), max(x0,x1)))
            x0 = x1
        else:
            verticals[x0].append((min(y0,y1), max(y0,y1)))
            y0 = y1
    return horizontals, verticals

In [22]:

def find_overlaps(a, b):
    overlaps = []
    for i, a_ranges in a.items():
        b_ranges = b.get(i)
        if not b_ranges:
            continue
        for a_start, a_end in a_ranges:
            overlaps = []
            di = abs(i)
            for b_start, b_end in b_ranges:
                if b_start <= a_start <= b_end:
                    overlaps.append(di + abs(a_start))
                if b_start <= a_end <= b_end:
                    overlaps.append(di + abs(a_end))
                if a_start < b_end < a_end:
                    overlaps.append(di + abs(b_end))
                if a_start < b_start < a_end:
                    overlaps.append(di + abs(b_start))
    return overlaps
    
def find_crosses(a, b):
    crosses = []
    for i, a_ranges in a.items():
        for j, b_ranges in b.items():
            for a_start, a_end in a_ranges:
                if j > a_end or j < a_start:
                    continue
                for b_start, b_end in b_ranges:
                    if b_start <= i <= b_end:
                        crosses.append(abs(i) + abs(j))
    return crosses
    
def solve(a, b):
    ah, av = parse_path(a)
    bh, bv = parse_path(b)
    return min([
        n
        for n in (
            find_overlaps(ah, bh) +
            find_overlaps(av, bv) +
            find_crosses(ah, bv) +
            find_crosses(av, bh)
        )
        if n
    ])


In [23]:
solve('R8,U5,L5,D3', 'U7,R6,D4,L4')

6

In [25]:
solve(a,b)

1195

In [31]:
# part 2

from collections import defaultdict

def parse_path(string):
    horizontals = defaultdict(list)
    verticals = defaultdict(list)
    path = string.split(',')
    x0, y0, x1, y1, steps = [0] * 5
    for leg in path:
        d = leg[0]
        val = int(leg[1:])
        if d == 'R':
            x1 = x0 + val
        elif d == 'U':
            y1 = y0 + val
        elif d == 'L':
            x1 = x0 - val
        elif d == 'D':
            y1 = y0 - val
        else:
            raise Exception('unknown direction')
        if y0 == y1:
            if x0 > x1:
                steps += val
                segment = (x1, x0, steps, -1)
            else:
                segment = (x0, x1, steps, 1)
                steps += val
            horizontals[y0].append(segment)
            x0 = x1
        else:
            if y0 > y1:
                steps += val
                segment = (y1, y0, steps, -1)
            else:
                segment = (y0, y1, steps, 1)
                steps += val
            verticals[x0].append(segment)
            y0 = y1
    return horizontals, verticals


def dist(start, base, delta, point):
    return base + delta * (point - start)


def find_overlaps(a, b):
    overlaps = []
    for i, a_ranges in a.items():
        b_ranges = b.get(i)
        if not b_ranges:
            continue
        for a_start, a_end, a_base, a_delta in a_ranges:
            overlaps = []
            di = abs(i)
            for b_start, b_end, b_base, b_delta in b_ranges:
                def _dist(pt):
                    return dist(a_start, a_base, a_delta, pt) + dist(b_start, b_base, b_delta, pt)
                if b_start <= a_start <= b_end:
                    overlaps.append(_dist(a_start))
                if b_start <= a_end <= b_end:
                    overlaps.append(_dist(a_end))
                if a_start < b_end < a_end:
                    overlaps.append(_dist(b_end))
                if a_start < b_start < a_end:
                    overlaps.append(_dist(b_start))
    return overlaps
    
def find_crosses(a, b):
    crosses = []
    for i, a_ranges in a.items():
        for j, b_ranges in b.items():
            for a_start, a_end, a_base, a_delta in a_ranges:
                if j > a_end or j < a_start:
                    continue
                for b_start, b_end, b_base, b_delta in b_ranges:
                    if b_start <= i <= b_end:
                        crosses.append(dist(a_start, a_base, a_delta, j) + dist(b_start, b_base, b_delta, i))
    return crosses
    
def solve(a, b):
    ah, av = parse_path(a)
    bh, bv = parse_path(b)
    return min([
        n
        for n in (
            find_overlaps(ah, bh) +
            find_overlaps(av, bv) +
            find_crosses(ah, bv) +
            find_crosses(av, bh)
        )
        if n
    ])


In [32]:
solve(a,b)

91518